In [3]:
!pip3 install keras

    100% |################################| 307kB 1.7MB/s ta 0:00:01
    100% |################################| 276kB 2.5MB/s ta 0:00:01
    100% |################################| 51kB 264kB/s ta 0:00:011
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built pyyaml


In [4]:
!pip3 install bs4

    100% |################################| 92kB 369kB/s ta 0:00:011
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [5]:
from urllib.request import urlopen
import urllib.request
import urllib
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import sys, os.path, re, csv
import pandas as pds
import numpy as np
from datetime import datetime
import codecs
import zipfile

In [4]:
# Setting work directory and opening csv files.
base_url = "http://www.aozora.gr.jp/"
data_dir = "./"
aozora_dir = data_dir + "aozora_data/"
log_dir = aozora_dir + "log/"
target_author_file = data_dir + "target_author.csv"

auth_target = []
with open(target_author_file,"r") as f:
    reader = csv.reader(f)
    for row in reader:
        auth_target.append(row)
auth_target

[['author', ' "url"'],
 ['natsume',
  ' http://www.aozora.gr.jp/index_pages/person148.html#sakuhin_list_1'],
 ['akutagawa',
  ' http://www.aozora.gr.jp/index_pages/person879.html#sakuhin_list_1'],
 ['mori',
  ' http://www.aozora.gr.jp/index_pages/person129.html#sakuhin_list_1'],
 ['sakaguchi',
  ' http://www.aozora.gr.jp/index_pages/person1095.html#sakuhin_list_1'],
 ['yoshikawa',
  ' http://www.aozora.gr.jp/index_pages/person1562.html#sakuhin_list_1']]

In [7]:
# Make directories for authors and csv, text extraction, and utf converted directories for them.

def make_workdir(aozora_dir=aozora_dir, auth_target=auth_target):
    if not os.path.exists(aozora_dir):
        try:
            os.makedirs(aozora_dir)
            print ("make: " + aozora_dir)
        except OSError as e:
            print (e)
            
    if not os.path.exists(log_dir):
        try:
            os.makedirs(log_dir)
            print ("make: " + log_dir)
        except OSError as e:
            print (e)
        
    for w in auth_target[1:]:
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        if not os.path.exists(auth_dir):
            try:
                os.makedirs(auth_dir)
                print ("make: " + auth_dir)
            except OSError as e:
                print (e)
        if not os.path.exists(auth_dir + "csv/"):
            try:
                os.makedirs(auth_dir + "csv/")
                print ("make: " + auth_dir + "csv/")
            except OSError as e:
                print (e)
        if not os.path.exists(auth_dir + "ext/"):
            try:
                os.makedirs(auth_dir + "ext/")
                print ("make: " + auth_dir + "ext/")
            except OSError as e:
                print (e)
        if not os.path.exists(auth_dir + "utf/"):
            try:
                os.makedirs(auth_dir + "utf/")
                print ("make: " + auth_dir + "utf/")
            except OSError as e:
                print (e)

In [8]:
# Downloads all the zip files from author's written pieces.

def download_zip(auth_target=auth_target):
    for w in auth_target[1:]:
        print ("starting %s" % w[0])
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        url = w[1]
        
        html = urlopen(url)

        if html.getcode() == 200:
            soup = BeautifulSoup(html, "lxml")
            piece_list = soup.find("ol")
            piece_links = piece_list.find_all("a")
            piece_links_np = np.array([["datetime","title","url","zip"]])
            for i in piece_links:
                title = i.string
                link = base_url + i["href"].replace("../", "")
                if "cards" in link:
                    print ("    piece: %s for %s" % (title, link))
                    piece_html = urlopen(link)
                    if piece_html.getcode() == 200:
                        soup = BeautifulSoup(piece_html, "lxml")
                        zip_part = soup.find_all("a", href=re.compile(".zip"))
                        if zip_part != []:
                            zip_file = zip_part[0]["href"]
                            zip_url = urllib.parse.urljoin(link, zip_file)
                            print ("        zip_url: %s" % zip_url)
                            now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
                            tmp = np.array([[now, title, link, zip_url]])
                            piece_links_np = np.vstack((piece_links_np, tmp))

                            file_name = os.path.basename(zip_url)
                            file_full_path = '{}{}'.format(auth_dir, file_name)
                            urllib.request.urlretrieve(zip_url, filename=file_full_path)

            piece_links_pds = pds.DataFrame(piece_links_np[1:,:], columns=piece_links_np[0,:])
            piece_links_pds.to_csv(log_dir + w[0] + '_dl_log.csv', quoting=csv.QUOTE_ALL)
        print ("finished %s" % w[0])

In [9]:
# Extract zip files to txt files. Its character encoding is in SHIfT-JIS.

def zip_extract(auth_target=auth_target):
    log_np = np.array([["datetime", "author", "zip"]])
    for w in auth_target[1:]:
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        ext_dir = '{}{}'.format(auth_dir, "ext/")
        files = os.listdir(auth_dir)
        for file in files:
            if "zip" in file:
                file_fullpath = auth_dir + file
                with zipfile.ZipFile(file_fullpath, 'r') as zip_file:
                    zip_file.extractall(path=ext_dir)
                    now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
                    tmp = np.array([[now, w[0], zip_file]])
                    log_np = np.vstack((log_np, tmp))
                    print ("extracted: " + str(zip_file))
        zip_ext_pds = pds.DataFrame(log_np[1:,:], columns=log_np[0,:])
        zip_ext_pds.to_csv(log_dir + w[0] + '_zip_log.csv', quoting=csv.QUOTE_ALL)

In [10]:
# Making txt files with SHIFT-JIS converted to UTF-8.

def convert_sjis_to_utf8(auth_target=auth_target):
    log_np = np.array([["datetime", "author", "file"]])
    for w in auth_target[1:]:
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        ext_dir = '{}{}'.format(auth_dir, "ext/")
        utf_dir = '{}{}'.format(auth_dir, "utf/")
        files = os.listdir(ext_dir)
        for file in files:
            if "txt" in file:
                file_name = ext_dir + file
                save_name = utf_dir + file
                fout = codecs.open(file_name, 'r', 'shift_jis')
                fsave = codecs.open(save_name, 'w+', 'utf-8')
                try:
                    for row in fout:
                        fsave.write(row)
                except Exception as e:
                    print (file + "gets exception: " + str(type(e)))
                finally:
                    fout.close()
                    fsave.close()
                    print ("converted: " + save_name)      
                    now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
                    tmp = np.array([[now, w[0], file]])
                    log_np = np.vstack((log_np, tmp))              
        convert_pds = pds.DataFrame(log_np[1:,:], columns=log_np[0,:])
        convert_pds.to_csv(log_dir + w[0] + '_cvt_log.csv', quoting=csv.QUOTE_ALL)

In [11]:
# Cleansing UTF-8 texts and convert them to files to CSV.

def data_cleanse(auth_target=auth_target):
    for w in auth_target[1:]:
        print ("starting: " + w[0])
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        ext_dir = '{}{}'.format(auth_dir, "ext/")
        utf_dir = '{}{}'.format(auth_dir, "utf/")
        csv_dir = '{}{}'.format(auth_dir, "csv/")
        files = os.listdir(utf_dir)
        for file in files:
            if "txt" in file:
                print ("     file: " + file)
                file_name = utf_dir + file
                np_lines = np.array([["auth","piece","line"]])
                f = open(file_name, 'r')

                lines = f.read()
                f.close
                
                lines = lines.replace(u'。', '。\n')
                lines = lines.split('\n')

                ruby = re.compile(u'\《.+?\》')
                chuki = re.compile(u'\［.+?\］')
                zen_sp = re.compile(u'　')
                zen_sp2 = re.compile(u'\u3000')

                for line in lines:
                    line_mod = ruby.sub("", line)
                    line_mod = chuki.sub("", line_mod)
                    line_mod = zen_sp.sub("", line_mod)
                    line_mod = zen_sp2.sub("", line_mod)
                    np_tmp = np.array([[w[0], file, line_mod]])
                    np_lines = np.vstack((np_lines, np_tmp))

                s_line = 1
                e_line = len(lines)
                np_lines_cut = np_lines[s_line:e_line,:]


                file = file.replace(".txt", "")
                lines_pds = pds.DataFrame(np_lines_cut, columns=np_lines[0,:])
                lines_pds.to_csv(csv_dir + file + '.csv', quoting=csv.QUOTE_ALL)

        print ("finished: " + w[0])

In [12]:
make_workdir(aozora_dir, auth_target)

make: ./aozora_data/
make: ./aozora_data/log/
make: ./aozora_data/natsume/
make: ./aozora_data/natsume/csv/
make: ./aozora_data/natsume/ext/
make: ./aozora_data/natsume/utf/
make: ./aozora_data/akutagawa/
make: ./aozora_data/akutagawa/csv/
make: ./aozora_data/akutagawa/ext/
make: ./aozora_data/akutagawa/utf/
make: ./aozora_data/mori/
make: ./aozora_data/mori/csv/
make: ./aozora_data/mori/ext/
make: ./aozora_data/mori/utf/
make: ./aozora_data/sakaguchi/
make: ./aozora_data/sakaguchi/csv/
make: ./aozora_data/sakaguchi/ext/
make: ./aozora_data/sakaguchi/utf/
make: ./aozora_data/yoshikawa/
make: ./aozora_data/yoshikawa/csv/
make: ./aozora_data/yoshikawa/ext/
make: ./aozora_data/yoshikawa/utf/


In [13]:
download_zip(auth_target)

starting natsume
    piece: イズムの功過 for http://www.aozora.gr.jp/cards/000148/card2314.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/2314_ruby_2291.zip
    piece: 一夜 for http://www.aozora.gr.jp/cards/000148/card1086.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/1086_ruby_5742.zip
    piece: 永日小品 for http://www.aozora.gr.jp/cards/000148/card758.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/758_ruby_6056.zip
    piece: 岡本一平著並画『探訪画趣』序 for http://www.aozora.gr.jp/cards/000148/card2669.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/2669_ruby_6341.zip
    piece: 思い出す事など for http://www.aozora.gr.jp/cards/000148/card792.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/792_ruby_2117.zip
    piece: カーライル博物館 for http://www.aozora.gr.jp/cards/000148/card1046.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/1046_ruby_4521.zip
    piece: 薤露行 for http://www.aozora.gr.jp/cards/000148/card769.html
 

        zip_url: http://www.aozora.gr.jp/cards/000148/files/42156_ruby_16023.zip
    piece: 点頭録 for http://www.aozora.gr.jp/cards/000148/card4672.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/4672_ruby_8553.zip
    piece: 『東洋美術図譜』 for http://www.aozora.gr.jp/cards/000148/card2313.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/2313_ruby_2287.zip
    piece: 道楽と職業 for http://www.aozora.gr.jp/cards/000148/card757.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/757_ruby_2986.zip
    piece: 長塚節氏の小説「土」 for http://www.aozora.gr.jp/cards/000148/card2682.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/2682_ruby_6333.zip
    piece: 中味と形式 for http://www.aozora.gr.jp/cards/000148/card788.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/788_ruby_2857.zip
    piece: 二百十日 for http://www.aozora.gr.jp/cards/000148/card751.html
        zip_url: http://www.aozora.gr.jp/cards/000148/files/751_ruby_1539.zip
    piece: 入社の

    piece: アグニの神 for http://www.aozora.gr.jp/cards/000879/card15.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/15_ruby_904.zip
    piece: 悪魔 for http://www.aozora.gr.jp/cards/000879/card3804.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/3804_ruby_27189.zip
    piece: 浅草公園 for http://www.aozora.gr.jp/cards/000879/card21.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/21_ruby_1427.zip
    piece: 兄貴のような心持 for http://www.aozora.gr.jp/cards/000879/card43361.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/43361_ruby_17690.zip
    piece: あの頃の自分の事 for http://www.aozora.gr.jp/cards/000879/card17.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/17_ruby_377.zip
    piece: あばばばば for http://www.aozora.gr.jp/cards/000879/card14.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/14_ruby_1261.zip
    piece: 鴉片 for http://www.aozora.gr.jp/cards/000879/card1138.html
        zip_url: http://www.aozora.gr

    piece: 解嘲 for http://www.aozora.gr.jp/cards/000879/card3764.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/3764_ruby_27200.zip
    piece: 蛙 for http://www.aozora.gr.jp/cards/000879/card3800.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/3800_ruby_27201.zip
    piece: 格さんと食慾 for http://www.aozora.gr.jp/cards/000879/card43365.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/43365_ruby_25702.zip
    piece: 影 for http://www.aozora.gr.jp/cards/000879/card64.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/64_ruby_1561.zip
    piece: 片恋 for http://www.aozora.gr.jp/cards/000879/card74.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/74_ruby_853.zip
    piece: かちかち山 for http://www.aozora.gr.jp/cards/000879/card3814.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/3814_ruby_27202.zip
    piece: 学校友だち for http://www.aozora.gr.jp/cards/000879/card3772.html
        zip_url: http://www.aozora.gr.

    piece: 校正後に for http://www.aozora.gr.jp/cards/000879/card89.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/89_ruby_1231.zip
    piece: 合理的、同時に多量の人間味 for http://www.aozora.gr.jp/cards/000879/card43376.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/43376_ruby_25698.zip
    piece: 黄粱夢 for http://www.aozora.gr.jp/cards/000879/card88.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/88_ruby_160.zip
    piece: 黒衣聖母 for http://www.aozora.gr.jp/cards/000879/card85.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/85_ruby_991.zip
    piece: 小杉未醒氏 for http://www.aozora.gr.jp/cards/000879/card43373.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/43373_ruby_25734.zip
    piece: 古千屋 for http://www.aozora.gr.jp/cards/000879/card82.html
        zip_url: http://www.aozora.gr.jp/cards/000879/files/82_ruby_1451.zip
    piece: 骨董羹 for http://www.aozora.gr.jp/cards/000879/card3738.html
        zip_url: http://www.aozora

...

    piece: 麓 for http://www.aozora.gr.jp/cards/001095/card45810.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/45810_ruby_38248.zip
    piece: 不良少年とキリスト for http://www.aozora.gr.jp/cards/001095/card42840.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/42840_ruby_24856.zip
    piece: ふるさとに寄する讃歌 for http://www.aozora.gr.jp/cards/001095/card45789.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/45789_ruby_38020.zip
    piece: ふるさとに寄する讃歌 for http://www.aozora.gr.jp/cards/001095/card45731.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/45731_ruby_24043.zip
    piece: 不連続殺人事件 for http://www.aozora.gr.jp/cards/001095/card42626.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/42626_ruby_59990.zip
    piece: フロオベエル雑感 for http://www.aozora.gr.jp/cards/001095/card42989.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/42989_ruby_21248.zip
    piece: 文学と国民生活 for http://www.aozora.gr.jp/cards/001095/c

    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/cards/001095/card43216.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43216_ruby_23300.zip
    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/cards/001095/card43217.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43217_ruby_23301.zip
    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/cards/001095/card43218.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43218_ruby_23302.zip
    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/cards/001095/card43219.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43219_ruby_23303.zip
    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/cards/001095/card43220.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43220_ruby_23304.zip
    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/cards/001095/card43221.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43221_ruby_23305.zip
    piece: 明治開化　安吾捕物 for http://www.aozora.gr.jp/car

    piece: 我が人生観 for http://www.aozora.gr.jp/cards/001095/card43202.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43202_ruby_22373.zip
    piece: わが精神の周囲 for http://www.aozora.gr.jp/cards/001095/card43168.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43168_ruby_33897.zip
    piece: わが戦争に対処せる工夫の数々 for http://www.aozora.gr.jp/cards/001095/card42854.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/42854_ruby_27002.zip
    piece: わが血を追ふ人々 for http://www.aozora.gr.jp/cards/001095/card42887.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/42887_ruby_24477.zip
    piece: 訣れも愉し for http://www.aozora.gr.jp/cards/001095/card45821.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/45821_ruby_34606.zip
    piece: 私の碁 for http://www.aozora.gr.jp/cards/001095/card43148.html
        zip_url: http://www.aozora.gr.jp/cards/001095/files/43148_ruby_29510.zip
    piece: 私の小説 for http://www.aozora.gr.jp/cards/001095/card428

        zip_url: http://www.aozora.gr.jp/cards/001562/files/55270_ruby_50499.zip
    piece: 私本太平記 for http://www.aozora.gr.jp/cards/001562/card52421.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52421_ruby_49660.zip
    piece: 私本太平記 for http://www.aozora.gr.jp/cards/001562/card52422.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52422_ruby_49670.zip
    piece: 私本太平記 for http://www.aozora.gr.jp/cards/001562/card52423.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52423_ruby_49680.zip
    piece: 私本太平記 for http://www.aozora.gr.jp/cards/001562/card52424.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52424_ruby_49690.zip
    piece: 私本太平記 for http://www.aozora.gr.jp/cards/001562/card52425.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52425_ruby_49698.zip
    piece: 私本太平記 for http://www.aozora.gr.jp/cards/001562/card52426.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52426_ruby_49700

    piece: 日本名婦伝 for http://www.aozora.gr.jp/cards/001562/card56497.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/56497_ruby_54171.zip
    piece: 日本名婦伝 for http://www.aozora.gr.jp/cards/001562/card56496.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/56496_ruby_54183.zip
    piece: 日本名婦伝 for http://www.aozora.gr.jp/cards/001562/card56059.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/56059_ruby_54180.zip
    piece: 日本名婦伝 for http://www.aozora.gr.jp/cards/001562/card56494.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/56494_ruby_54186.zip
    piece: 人間山水図巻 for http://www.aozora.gr.jp/cards/001562/card56058.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/56058_ruby_52987.zip
    piece: 野槌の百 for http://www.aozora.gr.jp/cards/001562/card52440.html
        zip_url: http://www.aozora.gr.jp/cards/001562/files/52440_ruby_50000.zip
    piece: 梅颸の杖 for http://www.aozora.gr.jp/cards/001562/card56495.html
    

In [14]:
zip_extract(auth_target)

extracted: <zipfile.ZipFile filename='./aozora_data/natsume/4686_ruby_9477.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/756_ruby_2948.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/4682_ruby_9463.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/751_ruby_1539.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/4684_ruby_9469.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/765_ruby_2469.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/796_ruby_2452.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/776_ruby_6020.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/757_ruby_2986.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/766_ruby_2068.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/natsume/2669_ruby_6341.zip' mode='r'>
extracted: <zipfi

extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3801_ruby_27221.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/135_ruby_765.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/188_ruby_1365.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3776_ruby_27235.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/4614_ruby_6776.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/58_ruby_1269.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/114_ruby_999.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/46_ruby_1431.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/42377_ruby_34744.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3807_ruby_27232.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/164_ruby_1455.zip' mod

extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3767_ruby_27191.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/2692_ruby_26930.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/4277_ruby_5489.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/85_ruby_991.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/181_ruby_1203.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/110_ruby_1195.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/2322_ruby_1511.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/56820_ruby_60835.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/45_ruby_1083.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/153_ruby_2349.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3784_ruby_27209.zip' m

extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/39_ruby_881.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/31_ruby_584.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/180_ruby_1300.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/66_ruby_400.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/24452_ruby_11237.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3782_ruby_27263.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/43382_ruby_17693.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/3750_ruby_27192.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/120_ruby_1011.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/49328_ruby_33094.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/akutagawa/4304_ruby_5519.zip' m

extracted: <zipfile.ZipFile filename='./aozora_data/mori/686_ruby_2541.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/2084_ruby_17360.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/2075_ruby_34226.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/674_ruby_15357.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/892_ruby_20935.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/2049_ruby_19718.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/432_ruby_22834.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/2547_ruby.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/681_ruby_22935.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/3614_ruby_12061.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/mori/688_ruby_23233.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozor

extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/56812_ruby_57364.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/43144_ruby_30147.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45918_ruby_36079.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42964_ruby_34468.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45743_txt_24011.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42829_ruby_26836.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45796_ruby_38019.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42849_ruby_33484.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/56814_ruby_57983.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/43151_ruby_29774.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi

extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42847_ruby_34827.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45789_ruby_38020.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45703_txt_23437.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/43215_ruby_23211.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42981_ruby_21011.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42990_txt_21249.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/43208_ruby_23204.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42922_ruby_23087.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45728_txt_24035.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42979_ruby_21238.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/4

extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45748_txt_23965.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/43248_ruby_24284.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42978_ruby_21237.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45734_ruby_23438.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/43167_ruby_33883.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45907_ruby_37368.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42960_ruby_30029.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45800_ruby_38012.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45844_ruby_60073.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45814_ruby_34566.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi

extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45868_ruby_33222.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45900_ruby_33803.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42617_ruby_29709.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45737_txt_23940.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42947_ruby_29845.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42914_ruby_23081.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42883_ruby_29790.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45803_ruby_34604.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/42909_ruby_23080.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi/45867_ruby_33221.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/sakaguchi

extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/52429_ruby_49706.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/56762_ruby_59510.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/55263_ruby_50532.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/56061_ruby_52992.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/58019_ruby_65428.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/56068_ruby_52997.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/52446_ruby_49980.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/56461_ruby_53134.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/52448_ruby_49982.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikawa/52438_ruby_49992.zip' mode='r'>
extracted: <zipfile.ZipFile filename='./aozora_data/yoshikaw

In [15]:
convert_sjis_to_utf8(auth_target)

converted: ./aozora_data/natsume/utf/watashino_kojinshugi.txt
converted: ./aozora_data/natsume/utf/sanzankoji.txt
converted: ./aozora_data/natsume/utf/gakushato_meiyo.txt
converted: ./aozora_data/natsume/utf/sorekara.txt
converted: ./aozora_data/natsume/utf/seiyoniwa_nai.txt
converted: ./aozora_data/natsume/utf/tsuchini_tsuite.txt
converted: ./aozora_data/natsume/utf/syojyosaku_tsuikaidan.txt
converted: ./aozora_data/natsume/utf/buncho.txt
converted: ./aozora_data/natsume/utf/kyoikuto_bungei.txt
converted: ./aozora_data/natsume/utf/omoidasu_kotonado.txt
converted: ./aozora_data/natsume/utf/masaoka_shiki.txt
converted: ./aozora_data/natsume/utf/nagatsuka_takashishino.txt
converted: ./aozora_data/natsume/utf/mon.txt
converted: ./aozora_data/natsume/utf/sorekara_yokoku.txt
converted: ./aozora_data/natsume/utf/tegami.txt
converted: ./aozora_data/natsume/utf/conrad.txt
converted: ./aozora_data/natsume/utf/kokoro_kokoku.txt
converted: ./aozora_data/natsume/utf/tentoroku.txt
converted: ./aozo

converted: ./aozora_data/akutagawa/utf/kachikachiyama.txt
converted: ./aozora_data/akutagawa/utf/shirami.txt
converted: ./aozora_data/akutagawa/utf/juensatsu.txt
converted: ./aozora_data/akutagawa/utf/onna.txt
converted: ./aozora_data/akutagawa/utf/kantei.txt
converted: ./aozora_data/akutagawa/utf/celt_no_hakumei.txt
converted: ./aozora_data/akutagawa/utf/karenosho.txt
converted: ./aozora_data/akutagawa/utf/nyotai.txt
converted: ./aozora_data/akutagawa/utf/kondo_koichiro.txt
converted: ./aozora_data/akutagawa/utf/aru_kyuyue_okuru.txt
converted: ./aozora_data/akutagawa/utf/hakkirishita_katachio.txt
converted: ./aozora_data/akutagawa/utf/yuwaku.txt
converted: ./aozora_data/akutagawa/utf/kottokan.txt
converted: ./aozora_data/akutagawa/utf/shishu.txt
converted: ./aozora_data/akutagawa/utf/koryomu.txt
converted: ./aozora_data/akutagawa/utf/soseki_sanbono_aki.txt
converted: ./aozora_data/akutagawa/utf/aruren'ai_shosetsu.txt
converted: ./aozora_data/akutagawa/utf/noroma_ningyo.txt
converted: 

converted: ./aozora_data/mori/utf/seinen.txt
converted: ./aozora_data/mori/utf/satoimono_me.txt
converted: ./aozora_data/mori/utf/okitsu_yagoemonno_isho.txt
converted: ./aozora_data/mori/utf/ogai_gyoshitowa_darezo.txt
converted: ./aozora_data/mori/utf/jiisan_baasan.txt
converted: ./aozora_data/mori/utf/kanzan_jittoku.txt
converted: ./aozora_data/mori/utf/hon'yakuni_tsuite.txt
converted: ./aozora_data/mori/utf/shokudo.txt
converted: ./aozora_data/mori/utf/yasui_fujin.txt
converted: ./aozora_data/mori/utf/shibue_chusai.txt
converted: ./aozora_data/mori/utf/eri.txt
converted: ./aozora_data/mori/utf/yuigon_sanshu.txt
converted: ./aozora_data/mori/utf/hashino_shita.txt
converted: ./aozora_data/mori/utf/yoshihisa_shinno_nenpu.txt
converted: ./aozora_data/mori/utf/resignation_no_setsu.txt
converted: ./aozora_data/mori/utf/sokkyo_shijin.txt
converted: ./aozora_data/mori/utf/natsume_soseki_ron.txt
converted: ./aozora_data/mori/utf/watashiga_jushigosaino_toki.txt
converted: ./aozora_data/mori/ut

converted: ./aozora_data/sakaguchi/utf/yamanokami_satsujin.txt
converted: ./aozora_data/sakaguchi/utf/12kodanshi_taijo.txt
converted: ./aozora_data/sakaguchi/utf/nobunaga_sakushano_kotoba.txt
converted: ./aozora_data/sakaguchi/utf/watasiwa_umio_dakishimete.txt
converted: ./aozora_data/sakaguchi/utf/suiri_shosetsuron.txt
converted: ./aozora_data/sakaguchi/utf/08_ango_aisai_monogatari.txt
converted: ./aozora_data/sakaguchi/utf/tsukihino_hanashi.txt
converted: ./aozora_data/sakaguchi/utf/shiganaoyani_bungakuno.txt
converted: ./aozora_data/sakaguchi/utf/01ango_shinnihonfudoki_kadai.txt
converted: ./aozora_data/sakaguchi/utf/watashino_go.txt
converted: ./aozora_data/sakaguchi/utf/sogano_abarenbo.txt
converted: ./aozora_data/sakaguchi/utf/18okami_daimyojin.txt
converted: ./aozora_data/sakaguchi/utf/akusairon.txt
converted: ./aozora_data/sakaguchi/utf/haha.txt
converted: ./aozora_data/sakaguchi/utf/meijikaika_ango_kojo.txt
converted: ./aozora_data/sakaguchi/utf/kenjutsuno_gokuio_kataru.txt
co

converted: ./aozora_data/sakaguchi/utf/sakonno_ikari.txt
converted: ./aozora_data/sakaguchi/utf/tegami_zatsudan.txt
converted: ./aozora_data/sakaguchi/utf/asuwa_tenkini_nare.txt
converted: ./aozora_data/sakaguchi/utf/waga_sensoni_taisho_seru_kufu.txt
converted: ./aozora_data/sakaguchi/utf/shisei_kandan.txt
converted: ./aozora_data/sakaguchi/utf/hansutairuno_ki.txt
converted: ./aozora_data/sakaguchi/utf/gendaino_sajutsu.txt
converted: ./aozora_data/sakaguchi/utf/nigetai_kokoro.txt
converted: ./aozora_data/sakaguchi/utf/kagero_dangi.txt
converted: ./aozora_data/sakaguchi/utf/seishinbyo_oboegaki.txt
converted: ./aozora_data/sakaguchi/utf/08ishino_shita.txt
converted: ./aozora_data/sakaguchi/utf/14maboroshino_to.txt
converted: ./aozora_data/sakaguchi/utf/ushi.txt
converted: ./aozora_data/sakaguchi/utf/sobomu.txt
converted: ./aozora_data/sakaguchi/utf/waga_kufuseru_ojiya.txt
converted: ./aozora_data/sakaguchi/utf/onna_kenshi.txt
converted: ./aozora_data/sakaguchi/utf/shindode_nakatta_ranbon

converted: ./aozora_data/yoshikawa/utf/tairanomasakado.txt
converted: ./aozora_data/yoshikawa/utf/shinsho_taikoki07.txt
converted: ./aozora_data/yoshikawa/utf/ashi.txt
converted: ./aozora_data/yoshikawa/utf/shinsho_taikoki06.txt
converted: ./aozora_data/yoshikawa/utf/hosokawa_garasha_fujin.txt
converted: ./aozora_data/yoshikawa/utf/chazuke_sanryaku.txt
converted: ./aozora_data/yoshikawa/utf/01jo_kenno_yonkunshi.txt
converted: ./aozora_data/yoshikawa/utf/07chihaya_jo.txt
converted: ./aozora_data/yoshikawa/utf/dainanko_fujin.txt


In [16]:
data_cleanse(auth_target)

starting: natsume
     file: watashino_kojinshugi.txt
     file: sanzankoji.txt
     file: gakushato_meiyo.txt
     file: sorekara.txt
     file: seiyoniwa_nai.txt
     file: tsuchini_tsuite.txt
     file: syojyosaku_tsuikaidan.txt
     file: buncho.txt
     file: kyoikuto_bungei.txt
     file: omoidasu_kotonado.txt
     file: masaoka_shiki.txt
     file: nagatsuka_takashishino.txt
     file: mon.txt
     file: sorekara_yokoku.txt
     file: tegami.txt
     file: conrad.txt
     file: kokoro_kokoku.txt
     file: tentoroku.txt
     file: sakubutsuno_hihyo.txt
     file: jinsei.txt
     file: densetsuno_jidai_jo.txt
     file: mohoto_dokuritsu.txt
     file: tayama_katai.txt
     file: baien_jo.txt
     file: hakase_mondaito_murdoch.txt
     file: michikusa.txt
     file: bokuno_mukashi.txt
     file: watashino_keikashita.txt
     file: shumino_iden.txt
     file: hakase_mondaino_nariyuki.txt
     file: tsuchini_tsuie.txt
     file: nakamito_keishiki.txt
     file: gendainihonno_kaika.t

     file: tanekono_yutsu.txt
     file: sato_haruoshino_koto.txt
     file: haruno_yo.txt
     file: aru_shakaishugisha.txt
     file: fugawarina_sakuhin.txt
     file: konparukai.txt
     file: zoku_basho_zakki.txt
     file: sennin.txt
     file: saigo_takamori.txt
     file: shujuno_kotobano_jo.txt
     file: shosetsuno_dokusha.txt
     file: shuchu.txt
     file: anokorono_jibun.txt
     file: mino_mawari.txt
     file: wasurerarenu_insho.txt
     file: chokoyuki.txt
     file: yarigatake_kiko.txt
     file: bunsho.txt
     file: pekin_nikki_sho.txt
     file: taisho_juninen.txt
     file: sobyo_sandai.txt
     file: takita_tetsutarokun.txt
     file: soteini.txt
     file: torokko.txt
     file: enju.txt
     file: aru_katakiuchino_hanashi.txt
     file: kesato_morito.txt
     file: sonokorono_akamonseikatsu.txt
     file: ronen.txt
     file: kimono.txt
     file: natsumesenseito_takitasan.txt
     file: kodokujigoku.txt
     file: susanono_mikoto.txt
     file: haru.txt
     fi

     file: shimabaranoran_zakki.txt
     file: gomanna_me.txt
     file: darakuron.txt
     file: 09akitaken_homonki.txt
     file: dentono_musansha.txt
     file: jo_nigetai_kokoro.txt
     file: shin'yawa_nemuruni_kagiru.txt
     file: tokaino_nakano_koto.txt
     file: watashino_soshiki.txt
     file: machiwa_furusato_sakusha.txt
     file: 09den'en_haremu.txt
     file: wagajinseikan_4.txt
     file: shogono_satsujin.txt
     file: jinsei_mittsuno_tanoshimi.txt
     file: chisana_yagino_kiroku.txt
     file: wagajinseikan_6.txt
     file: chohen_shosetsu_jihyo.txt
     file: jirorino_onna.txt
     file: zoku_darakuron.txt
     file: 05katsu_musui.txt
     file: gaitoto_aozora.txt
     file: kaze_hakase.txt
     file: kuroda_josui.txt
     file: omochabako.txt
     file: dosutoefusukito_baruzakku.txt
     file: chabanni_yosete.txt
     file: kanino_awa.txt
     file: hakuchi.txt
     file: shudanmiai.txt
     file: hanadakiyoteru_ron.txt
     file: keimano_genso.txt
     file: yokok

     file: fukuin_satsujin_jikenni_tsuite.txt
     file: ishikito_jikantono_kankei.txt
     file: kazeto_hikarito.txt
     file: bunkasai.txt
     file: oihirosuketoyu_otoko.txt
     file: tennoheikani_sasaguru.txt
     file: shobushi.txt
     file: nanpufu.txt
     file: 05kieuseta_sabaku.txt
     file: pittya_satsujin_jiken.txt
     file: geido_chini_otsu.txt
     file: kamisamao_undahitobito.txt
     file: dazaiosamu_jyoshiko.txt
     file: kobayashisanto_watashino.txt
     file: shito_hanauta.txt
     file: daini_geijutsu_ronni_tsuite.txt
     file: 01mayaku_jisatsu_shukyo.txt
     file: rinrakuno_seishun.txt
     file: tsuchino_naka.txt
     file: hinsezareba_donsu.txt
     file: koki_rohen_yawashu.txt
     file: ihojinni_tsuite.txt
     file: ikon.txt
     file: watashino_shosetsu.txt
     file: taiho_shugisha.txt
     file: shinjin'e.txt
     file: atarashiki_seikaku_kanjo.txt
     file: semi.txt
     file: shinshun_nihonno_sorao.txt
     file: tadano_bungaku.txt
     file: mura

     file: ookaechizen.txt
     file: 04hino_maki.txt
     file: minamotono_yoritomo.txt
     file: tani_tateki_fujin.txt
     file: tennoto_keiba.txt
     file: tazaki_sounto_sonoko.txt
     file: 12minatogawa_jo.txt
     file: 03mizuno_maki.txt
     file: kuse.txt
     file: oriorino_ki.txt
     file: 13kokubyaku_jo.txt
     file: jirokichi_koshi.txt
     file: otaka.txt
     file: natsumushi_andon.txt
     file: 06sorano_maki.txt
     file: kankanmushiwa_utau.txt
     file: 09kenmu_rakugaki_jo.txt
     file: rakujitsuno_sogonni_niru.txt
     file: shinda_chidori.txt
     file: ginga_matsuri.txt
     file: shinpen_chushingura.txt
     file: ningen_sansui_zumaki.txt
     file: yagyu_tsukikage_sho.txt
     file: shinsho_taikoki02.txt
     file: 02chino_maki.txt
     file: shinsho_taikoki03.txt
     file: 08nitta_jo.txt
     file: shosetsuno_tane.txt
     file: 09tonanno_maki.txt
     file: 07nitenno_maki.txt
     file: zocho_tenno.txt
     file: shinsho_taikoki01.txt
     file: uesugik

In [6]:
"""
This is data integrating and training script for author classification using Keras with character-level cnn.
Be sure to get the text files from Aozora-bunko and convert them to UTF-8 encoded csv files, just by running aozora-scrape.py.
"""

import sys, os.path, re, csv, os, glob
import pandas as pds
import numpy as np
import zipfile
import codecs

In [7]:
from keras.layers import Activation, Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, Reshape, Input, merge
from keras.models import Model, Sequential
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, Callback, CSVLogger, ModelCheckpoint

Using TensorFlow backend.


In [8]:
# Setting work directory and opening csv files.
base_url = "http://www.aozora.gr.jp/"
data_dir = "./"
aozora_dir = data_dir + "aozora_data/"
log_dir = aozora_dir + "log/"
target_author_file = data_dir + "target_author.csv"

auth_target = []
with open(target_author_file,"r") as f:
    reader = csv.reader(f)
    for row in reader:
        auth_target.append(row)
auth_target

[['author', ' "url"'],
 ['natsume',
  ' http://www.aozora.gr.jp/index_pages/person148.html#sakuhin_list_1'],
 ['akutagawa',
  ' http://www.aozora.gr.jp/index_pages/person879.html#sakuhin_list_1'],
 ['mori',
  ' http://www.aozora.gr.jp/index_pages/person129.html#sakuhin_list_1'],
 ['sakaguchi',
  ' http://www.aozora.gr.jp/index_pages/person1095.html#sakuhin_list_1'],
 ['yoshikawa',
  ' http://www.aozora.gr.jp/index_pages/person1562.html#sakuhin_list_1']]

In [9]:
# Generating integrated file of csv's of all the pieces for each author.
# One csv file is generated and saved for each author.

def author_data_integ(auth_target=auth_target):
    for w in auth_target[1:]:
        print ("starting: " + w[0])
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        csv_dir = '{}{}'.format(auth_dir, "csv/")
        files = os.listdir(csv_dir)
        integ_np = np.array([["author", "line"]])
        for file in files:
            if "csv" in file:
                print ("   now at: " + file)
                file_name = csv_dir + file
                pds_data = pds.read_csv(file_name, index_col=0)
                pds_data = pds_data.dropna()
                np_data = np.array(pds_data.ix[:,[0,2]])

                out = [j for j in range(len(np_data)) if '-----------' in str(np_data[j,1])]
                if not out: out = [1]
                hyphen_pos = int(out[len(out) - 1])

                last_20 = len(np_data) - 20

                np_data = np_data[hyphen_pos+1:last_20,:]
                integ_np = np.vstack((integ_np, np_data))

        integ_pds = pds.DataFrame(integ_np[1:,:], columns=integ_np[0,:])
        integ_pds.to_csv(auth_dir + w[0] + '_integ.csv', quoting=csv.QUOTE_ALL)
        print ("finished: " + w[0])

In [10]:
author_data_integ()

starting: natsume
   now at: shikino_e.csv
   now at: mankan_tokorodokoro.csv
   now at: kokoro_jijo.csv
   now at: meijizano_shokan.csv
   now at: bunshino_seikatsu.csv
   now at: nowaki.csv
   now at: kusamakura.csv
   now at: sanshiro.csv
   now at: nyusyano_ji.csv
   now at: maboroshino_tate.csv
   now at: yono_egakantosuru.csv
   now at: mudai.csv
   now at: higansugimade.csv


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


   now at: shoshuno_ichinichi.csv
   now at: doraku.csv
   now at: bungeiwa_danshi.csv
   now at: bungeino_tetsugaku.csv
   now at: henna_oto_kyu.csv
   now at: bocchan.csv
   now at: neko_gehen.csv
   now at: kotono_sorane.csv
   now at: ichiya.csv
   now at: london_shosoku.csv
   now at: bungeito_dotoku.csv
   now at: kofu.csv
   now at: teichono_isho.csv
   now at: hitaino_otokoo_yomu.csv
   now at: okamoto_ippei.csv
   now at: wagahaiwa_nekodearu.csv
   now at: yoto_mannenhitsu.csv
   now at: ismno_koka.csv
   now at: koeber_rsensei.csv
   now at: shaseibun.csv
   now at: glass_dono_uchi.csv
   now at: koeber_sensei.csv
   now at: gubijinso.csv
   now at: kyoshikun.csv
   now at: senso_ikichigai.csv
   now at: takahamakyosi_cho_keito.csv
   now at: kikokujino_ichiya.csv
   now at: nihyakutoka.csv
   now at: kyoni_tsukeru_yube.csv
   now at: neko_jyo.csv
   now at: sanshiro_yokoku.csv
   now at: eijitsu_shohin.csv
   now at: kokoro.csv
   now at: miekichiate_shokan.csv
   now at: ts

   now at: uwagoto.csv
   now at: akuma.csv
   now at: proletariat.csv
   now at: shosetsusaho.csv
   now at: furansu_bungakuto_boku.csv
   now at: umano_ashi.csv
   now at: kubota_mantaro.csv
   now at: jozetsu.csv
   now at: nagasaki.csv
   now at: sennin2.csv
   now at: yamashigi.csv
   now at: katakoi.csv
   now at: futari_komachi.csv
   now at: chuto.csv
   now at: geijutsu_sonota.csv
   now at: kabocha.csv
   now at: shuzanzu.csv
   now at: honyaku_shohin.csv
   now at: ichiban_kinorino_surutoki.csv
   now at: nenmatsuno_ichinichi.csv
   now at: mata_issetsu1.csv
   now at: jigokuhen_shinji.csv
   now at: omu.csv
   now at: fushodan.csv
   now at: honjo_ryogoku_shinjishinkana.csv
   now at: kaikano_otto.csv
   now at: zoku_bungeitekina.csv
   now at: koseigoni.csv
   now at: nijunengono_senso.csv
   now at: daidojishinsukeno_hansei.csv
   now at: majutsu.csv
   now at: yuri.csv
   now at: toyono_aki.csv
   now at: aruhino_oishi_kuranosuke.csv
   now at: hitorino_mumeisakka.csv
  

   now at: kasutorisha_jiken.csv
   now at: omocha_bako.csv
   now at: bungakuto_kokuminseikatsu.csv
   now at: anogo_shimodagaishi.csv
   now at: akirame_anego.csv
   now at: ramuneshino_koto.csv
   now at: gakudo_shoron.csv
   now at: kongono_jiinseikatsu.csv
   now at: sakurano_morino_mankaino.csv
   now at: nihon_seishin.csv
   now at: machiwa_furusato.csv
   now at: golfto_waruinakama.csv
   now at: uragiri.csv
   now at: sakkaronni_tsuite.csv
   now at: sanjussai.csv
   now at: ikuji.csv
   now at: nihonjinni_tsuite.csv
   now at: 17kazokuwa_rokunin.csv
   now at: nankinmushi_satsujinjiken.csv
   now at: atogaki_izukoe.csv
   now at: nichigetsusama.csv
   now at: kurotanimura.csv
   now at: ashinonaiotokoto_kubino.csv
   now at: teisono_habato_genkai.csv
   now at: kansokano_umarederu_tameni.csv
   now at: chihobunkano_kakuritsu.csv
   now at: sengo_gokakusha.csv
   now at: kinkyohokoku.csv
   now at: 08storippu_bato.csv
   now at: 02takachihoni_fuyuame_fureri.csv
   now at: maki

   now at: farce_ni_tsuite.csv
   now at: hamon.csv
   now at: gonimo_meijinsen.csv
   now at: piero_dendosha.csv
   now at: gendai_ninjutsuden.csv
   now at: atarashiki_bungaku.csv
   now at: watashino_tanteishosetsu.csv
   now at: fubuki_monogatari.csv
   now at: hidano_kao.csv
   now at: 07chio_miru_shinju.csv
   now at: 10fukumen_yashiki.csv
   now at: watashiwa_umio_dakishimete_itai.csv
   now at: yowa_benmeisu.csv
   now at: inchiki_bungaku_bokumetsu_zatsudan.csv
   now at: zenso.csv
   now at: zokubutsuseito_sakka.csv
   now at: fumoto.csv
   now at: hongono_namikimichi.csv
   now at: hanasakeru_ishi.csv
   now at: 04asukano_maboroshi.csv
   now at: 03_seishinbyo_shindansho.csv
   now at: shisei_satujinjiken.csv
   now at: kotanno_fukakuo_haisu.csv
   now at: igo_shugyo.csv
   now at: dokyo.csv
   now at: wagajinseikan_2.csv
   now at: 12inazumawa_mitari.csv
   now at: shiganaoyani_bungakuno.csv
   now at: tsukihino_hanashi.csv
   now at: sogano_abarenbo.csv
   now at: watashino

   now at: jitenno_susume.csv
   now at: narutohicho05.csv
   now at: 02yagyu_sekishusai.csv
   now at: hanakami_ronin.csv
   now at: narutohicho04.csv
   now at: 10kazabana_jo.csv
   now at: kagaribino_onna.csv
finished: yoshikawa


In [11]:
# Loading integrated csv files.

def load_integ(author, auth_dir):
    integ_csv = auth_dir + author + "_integ.csv"
    data = pds.read_csv(integ_csv)
    return data

In [12]:
natsume_data = load_integ(auth_target[1][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[1][0]))
np_natsume = np.array(natsume_data.ix[1:,1:])

akutagawa_data = load_integ(auth_target[2][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[2][0]))
np_akutagawa = np.array(akutagawa_data.ix[1:,1:])

mori_data = load_integ(auth_target[3][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[3][0]))
np_mori = np.array(mori_data.ix[1:,1:])

sakaguchi_data = load_integ(auth_target[4][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[4][0]))
np_sakaguchi = np.array(sakaguchi_data.ix[1:,1:])

yoshikawa_data = load_integ(auth_target[5][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[5][0]))
np_yoshikawa = np.array(yoshikawa_data.ix[1:,1:])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.l

In [13]:
# Preparing arrays

natsume_txt = np.array([np_natsume[:,1]]).T
akutagawa_txt = np.array([np_akutagawa[:,1]]).T
mori_txt = np.array([np_mori[:,1]]).T
sakaguchi_txt = np.array([np_sakaguchi[:,1]]).T
yoshikawa_txt = np.array([np_yoshikawa[:,1]]).T

In [14]:
natsume_id = np.array([np.zeros(len(np_natsume))]).T
akutagawa_id = np.array([np.zeros(len(np_akutagawa)) + 1]).T
mori_id = np.array([np.zeros(len(np_mori)) + 2]).T
sakaguchi_id = np.array([np.zeros(len(np_sakaguchi)) + 3]).T
yoshikawa_id = np.array([np.zeros(len(np_yoshikawa)) + 4]).T

In [15]:
natsume = np.hstack((natsume_txt, natsume_id))
akutagawa = np.hstack((akutagawa_txt, akutagawa_id))
mori = np.hstack((mori_txt, mori_id))
sakaguchi = np.hstack((sakaguchi_txt, sakaguchi_id))
yoshikawa = np.hstack((yoshikawa_txt, yoshikawa_id))

In [16]:
print (len(natsume), len(akutagawa), len(mori), len(sakaguchi), len(yoshikawa))

105372 63504 105481 147463 417000


In [17]:
"""
This time, I am using pieces from Soseki Natsume, Ryunosuke Akutagawa, Ogai Mori and Ango Sakaguchi
for they have about the same number of lines.
Omitting Eiji Yoshikawa due to the fact that he has three time more line of texts than others.
"""

'\nThis time, I am using pieces from Soseki Natsume, Ryunosuke Akutagawa, Ogai Mori and Ango Sakaguchi\nfor they have about the same number of lines.\nOmitting Eiji Yoshikawa due to the fact that he has three time more line of texts than others.\n'

In [18]:
data_integ = np.vstack((np.vstack((np.vstack((natsume, akutagawa)),mori)),sakaguchi))

In [19]:
# Converting each line of texts to encoded array.

def load_data(txt, max_length=200):
    txt_list = []
    for l in txt:
        txt_line = [ord(x) for x in str(l).strip()]
        # You will get encoded text in array, just like this
        # [25991, 31456, 12391, 12399, 12394, 12367, 12387, 12390, 23383, 24341, 12391, 12354, 12427, 12290]
        txt_line = txt_line[:max_length]
        txt_len = len(txt_line)
        if txt_len < max_length:
            txt_line += ([0] * (max_length - txt_len))
        txt_list.append((txt_line))
    return txt_list

In [20]:
# Making arrays for training text and target author
txt_list = load_data(txt = data_integ[:,0])
np_txt = np.array(txt_list)

tgt_list = data_integ[:,1]
np_tgt_list = np_utils.to_categorical(tgt_list)

In [21]:
# Creating character-level convolutional neural network model.

def create_model(embed_size=128, max_length=200, filter_sizes=(2, 3, 4, 5), filter_num=64):
    inp = Input(shape=(max_length,))
    emb = Embedding(0xffff, embed_size)(inp)
    emb_ex = Reshape((max_length, embed_size, 1))(emb)
    convs = []
    for filter_size in filter_sizes:
        conv = Convolution2D(filter_num, filter_size, embed_size, activation="relu")(emb_ex)
        pool = MaxPooling2D(pool_size=(max_length - filter_size + 1, 1))(conv)
        convs.append(pool)
    convs_merged = merge(convs, mode='concat')
    reshape = Reshape((filter_num * len(filter_sizes),))(convs_merged)
    fc1 = Dense(64, activation="relu")(reshape)
    bn1 = BatchNormalization()(fc1)
    do1 = Dropout(0.5)(bn1)
    fc2 = Dense(4, activation='sigmoid')(do1)
    model = Model(input=inp, output=fc2)
    return model

In [22]:
# Training the model.

def train(inputs, targets, batch_size=100, epoch_count=100, max_length=200, model_filepath=aozora_dir + "model.h5", learning_rate=0.001):
    start = learning_rate
    stop = learning_rate * 0.01
    learning_rates = np.linspace(start, stop, epoch_count)

    model = create_model(max_length=max_length)
    optimizer = Adam(lr=learning_rate)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()

    target = os.path.join('/tmp', 'weights.*.hdf5')
    files = [(f, os.path.getmtime(f)) for f in glob.glob(target)]
    if len(files) != 0:
        latest_saved_model = sorted(files, key=lambda files: files[1])[-1]
        model.load_weights(latest_saved_model[0])
    
    # Logging file for each epoch
    csv_logger_file = '/tmp/clcnn_training.log'
    
    # Checkpoint model for each epoch
    checkpoint_filepath = "/tmp/weights.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5"

    model.fit(inputs, targets,
              nb_epoch=epoch_count,
              batch_size=batch_size,
              verbose=1,
              validation_split=0.1,
              shuffle=True,
              callbacks=[
                  LearningRateScheduler(lambda epoch: learning_rates[epoch]),
                  CSVLogger(csv_logger_file),
                  ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True, save_weights_only=False, monitor='val_acc')
              ])

    model.save(model_filepath)

In [23]:
train(np_txt, np_tgt_list)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 128), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 128), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 128), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 128), activation="relu")`
  if __name__ == '__main__':


TypeError: 'module' object is not callable